# Data Fetching

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
# Get the S&P 500 tickers from Wikipedia
sp500url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
data_table = pd.read_html(sp500url)
tickers = [ticker.replace('.', '-') for ticker in data_table[0]['Symbol']]
# Download S&P 500 data
sp_500 = yf.download(tickers, start='2005-01-01', end='2023-06-30')['Adj Close']
# Download specific tickers with '-' in their symbols
yf.download('BF-B')['Adj Close']
yf.download('BRK-B')['Adj Close']
sp_500.to_csv("sp500_data.csv")
# Convert the first column (index 0) into a list.
column_list = sp_500.columns.tolist()
# Get sector information from column list and merge it with company codes
sectors = [yf.Ticker(ticker).info.get("sector", "Unknown") for ticker in column_list]
# Shorten the name of sectors
shortened_sectors = ["".join([word[0].upper() for word in sector.split()]) if ' ' in sector else sector[:2].upper() for sector in sectors]
#Merge the sector Name with company codes
merged_data = [f"{sector}_{ticker}" for sector, ticker in zip(shortened_sectors, column_list)]
sp_500.columns = merged_data
# Sort columns alphabetically and then numerically
sp500_sorted = sp_500.reindex(sorted(sp_500.columns, key=lambda s: ("".join([c for c in s if c.isalpha()]), int("".join([c for c in s if c.isdigit()])) if any(c.isdigit() for c in s) else 0)), axis=1)
# Function to remove consecutive NaN values
def remove_consecutive_nan(df, threshold):
    mask = df.notna().cumsum().mask(df.notna()).apply(pd.Series.value_counts).gt(threshold)
    return df.loc[:, ~mask.any(axis=0)]
# Remove consecutive NaN values and save to CSV
sp500_nan = remove_consecutive_nan(sp500_sorted, 3)
sp500_nan.to_csv("sp500_nan.csv")
# Fill NaN values with the mean
sp500_nan.fillna(sp500_nan.mean(), inplace=True)
# Calculate log returns
log_returns = np.log(sp500_nan) - np.log(sp500_nan.shift(1))
returns = log_returns.iloc[1:]
# Drop columns where all values are greater than or equal to 0.8
drop = returns
drop = drop.loc[:, drop.max() < 0.8]
print(drop)
drop.to_csv("sp500_drop0.8.csv")


[*********************100%%**********************]  503 of 503 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


<ipython-input-30-000ebeaaa722>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp500_nan.fillna(sp500_nan.mean(), inplace=True)
<ipython-input-30-000ebeaaa722>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp500_nan.fillna(sp500_nan.mean(), inplace=True)
<ipython-input-30-000ebeaaa722>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

              BM_ALB    BM_APD     BM_DD    BM_ECL    BM_EMN    BM_FCX  \
Date                                                                     
2005-01-04 -0.041588 -0.019307 -0.021871 -0.012798 -0.093410 -0.040307   
2005-01-05 -0.024406  0.000878 -0.010178 -0.009707 -0.011175 -0.011538   
2005-01-06  0.019186  0.006122  0.011417  0.002952  0.020710 -0.001416   
2005-01-07  0.003574  0.005392 -0.009540 -0.005913 -0.008194  0.009591   
2005-01-10 -0.007437  0.000694  0.014686  0.008855  0.002102 -0.005348   
...              ...       ...       ...       ...       ...       ...   
2023-06-23 -0.026090 -0.006095 -0.000885 -0.009608  0.001656 -0.034579   
2023-06-26  0.022632  0.000454  0.000295  0.012551  0.026879  0.023185   
2023-06-27  0.008181  0.017377  0.020439  0.002823  0.012562  0.028616   
2023-06-28 -0.045361 -0.000309  0.006769  0.000939  0.009379 -0.029890   
2023-06-29  0.009083  0.011705  0.016936  0.015344  0.005200  0.003309   

              BM_FMC    BM_IFF    BM_

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
